In [72]:
# Import Libraries

import pandas as pd
import requests
import os
import datetime

In [73]:
# setting my keys & tokens using environment variables, using a YAML file would work as well

consumer_key = os.environ["TWITTER_API_KEY"]
consumer_secret = os.environ["TWITTER_API_SECRET"]
access_token = os.environ["TWITTER_ACCESS_TOKEN"]
access_token_secret = os.environ["TWITTER_SECRET_TOKEN"]
bearer_token = os.environ["TWITTER_BEARER"]

In [74]:
# function to format the bearer token for request purposes

def connect_to_twitter():
    bearer_token = os.environ["TWITTER_BEARER"]
    return {"Authorization":"Bearer {}".format(bearer_token)}

In [75]:
headers = connect_to_twitter()

In [66]:
# function to make request to url endpoint
# url can be changed based on the endpoint you want to connect to
# params can be changed based on endpoint
# query within params can be changed to your specific search

def make_request(headers, start_date, end_date):
    url = "https://api.twitter.com/2/tweets/search/recent"
    params = "query=(TSLA OR TESLA)&tweet.fields=public_metrics,created_at&start_time={}&end_time={}&max_results=100".format(start_date, end_date)
    return requests.request("GET", url, params=params, headers=headers).json()

In [44]:
# function to create a standard df based on the features needed
# columns can be altered to the features you are looking for

def make_df(response):
    return pd.DataFrame(response['data'], columns=['id','created_at', 'text', 'public_metrics'])

In [69]:
# function to populate a csv file with tweets pulled from api
# since the api endpoint I'm using (/search/recent) is only available within the last week the day variable 
#     is set to 6 days previously and works closer to today with each loop

def populate_tweets():    
    day = (pd.to_datetime(datetime.date.today() - datetime.timedelta(days=6))).day
    tweet_df = pd.read_csv('data/TSLA_tweets.csv')
#     tweet_df.to_csv('data/test.csv')
    for i in range(6):
        date = '{}/03/2021'.format(day)
        date1 = '{}/03/2021'.format(day+1)
        start_date = pd.to_datetime(date, format='%d/%m/%Y').isoformat("T")+"Z"
        end_date = pd.to_datetime(date1, format='%d/%m/%Y').isoformat("T")+"Z"
        response = make_request(headers, start_date, end_date)
        tmp_df = make_df(response)
        tweet_df = pd.concat([tweet_df, tmp_df], ignore_index=True)
        day+=1
    tweet_df.to_csv('data/TSLA_tweets.csv')

In [70]:
populate_tweets()